In [1]:
import numpy as np
import pandas as pd
import xarray as xr
import matplotlib.pyplot as plt
import seaborn as sns
from pathlib import Path
from tqdm.auto import tqdm
import pickle

from lisfloodreservoirs import Config, read_attributes, read_timeseries
from lisfloodreservoirs.calibration import read_results

In [2]:
# from lisfloodreservoirs.models.camaflood import Camaflood
# from lisfloodreservoirs.calibration.camaflood import Camaflood_calibrator

In [3]:
from lisfloodreservoirs.models import get_model
from lisfloodreservoirs.calibration import get_calibrator

In [4]:
from lisfloodreservoirs.utils.utils import return_period

In [5]:
from lisfloodreservoirs.utils.metrics import KGEmod#, is_pareto_efficient
from lisfloodreservoirs.utils.plots import plot_pareto_front

In [6]:
from scipy.stats import ttest_rel
from itertools import combinations
from statsmodels.stats.multitest import multipletests

## Configuration

In [7]:
path_tests = Path('Z:/nahaUsers/casadje/datasets/reservoirs/ResOpsUS/v2.1/test')
cfg = Config(path_tests / 'camaflood_2var_1000_8ngs.yml')

In [8]:
runs = [
    '1000_4',
    '1000_8',
    '2000_4',
    '2000_8',
    '10000_16'
]

## Data

### Reservoirs

In [9]:
# list of reservoirs to be trained
reservoir_list = pd.read_csv(cfg.RESERVOIRS_FILE, header=None).squeeze().tolist()

# import all tables of attributes
attributes = read_attributes(cfg.PATH_DATA / 'attributes', reservoir_list)

# invert the one-hot encoder of main reservoir use
col_use = attributes.columns[attributes.columns.str.contains('MAIN_')]
col_use = [col for col in col_use if 'BASIN' not in col]
main_use = attributes[col_use].idxmax(axis=1)
attributes['MAIN_USE'] = main_use.replace({col: col.split('_')[-1] for col in col_use})
attributes.drop(col_use, axis=1, inplace=True)

print(f'{len(attributes)} reservoirs in the attributes table')

164 reservoirs in the attributes table


### Time series

In [10]:
# training periods
with open(cfg.PERIODS_FILE, 'rb') as file:
    periods = pickle.load(file)
# periods = {str(ID): {f'{x}_dates': [date] for x, date in dct.items()} for ID, dct in periods.items()}

# read time series
inputs = [var for var in [cfg.INFLOW, cfg.PRECIPITATION, cfg.EVAPORATION, cfg.DEMAND] if var]
outputs = ['storage', 'outflow']
timeseries = read_timeseries(
    path=cfg.PATH_DATA / 'time_series' / 'csv',
    reservoirs=attributes.index,
    periods=periods,
    variables=inputs + outputs
)
print(f'{len(timeseries)} reservoirs in the time series')

# compute 100-year return period of the inflow
Q100 = [return_period(ts.inflow, T=100) for grand_id, ts in timeseries.items()]
Q100 = xr.DataArray(data=Q100, coords=[list(timeseries)], dims=['ID'])

  0%|          | 0/164 [00:00<?, ?it/s]

164 reservoirs in the time series


### Calibration

### Old convergence criteria

In [11]:
subpath_calib = Path('camaflood/calibration_old_convergence/bivariate')
results_old = {grand_id: {} for grand_id in reservoir_list}
for grand_id, dct in tqdm(results_old.items(), desc='reservoirs'):
    for run in runs:
        path = path_tests / run / subpath_calib
        file = path / f'{grand_id}_samples.csv'
        if file.is_file():
            iterations, parameters = read_results(file)
            iterations.chain = iterations.chain.astype(int)
            iterations['KGE'] = 1 - iterations.like1
            # iterations.drop('like1', axis=1, inplace=True)
            dct[run] = iterations

reservoirs:   0%|          | 0/164 [00:00<?, ?it/s]

### New convergence criteria

In [12]:
subpath_calib = Path('camaflood/calibration/bivariate')
results = {grand_id: {} for grand_id in reservoir_list}
for grand_id, dct in tqdm(results.items(), desc='reservoirs'):
    for run in runs:
        path = path_tests / run / subpath_calib
        file = path / f'{grand_id}_samples.csv'
        if file.is_file():
            iterations, parameters = read_results(file)
            iterations.chain = iterations.chain.astype(int)
            iterations['KGE'] = 1 - iterations.like1
            # iterations.drop('like1', axis=1, inplace=True)
            dct[run] = iterations

reservoirs:   0%|          | 0/164 [00:00<?, ?it/s]

## Analysis

### Single reservoir

In [13]:
# grand_id = 612

# fig, ax = plt.subplots(figsize=(16, 4))
# ax.set(
#     xlabel='iterations',
#     ylabel='KGE',
#     ylim=(-1, 1.05)
# )

# for run, iterations in results[grand_id].items():
#     iter_best, kge_best = iterations.KGE.idxmax(), iterations.KGE.max()
#     print('{0:>8}\tKGE = {1:.3f} (iteration {2})'.format(run, kge_best, iter_best))
#     ax.plot(iterations.KGE, lw=.8, label='run')

### Summary best performance

#### Old convergence criteria

In [ ]:
# extract best performance for each reservoir and calibration setup
summary_old = pd.DataFrame(index=reservoir_list, columns=runs, dtype=float)
for grand_id, dct in results_old.items():
    for run, iterations in dct.items():
        summary_old.loc[grand_id, run] = iterations.KGE.max()

In [ ]:
# compare performance
fig, ax = plt.subplots(figsize=(8, 5))
sns.boxplot(
    summary_old,
    width=.5,
    showcaps=False,
    showfliers=False,
    boxprops=dict(facecolor='none', edgecolor='k', linewidth=1),
    whiskerprops=dict(color='k', linewidth=1),
    medianprops=dict(color='k', linewidth=2),
    zorder=2,
    ax=ax
)
sns.swarmplot(summary_old, ax=ax, zorder=1)
ax.set(
    ylim=(-1.05, 1.05),
    ylabel='KGE',
    title='CaMa-Flood',
)
ax.spines['left'].set_bounds([-1, 1])
ax.spines[['top', 'right', 'bottom']].set_visible(False)
ax.tick_params(axis='x', length=0);

#### New convergence criteria

In [ ]:
# extract best performance for each reservoir and calibration setup
summary = pd.DataFrame(index=reservoir_list, columns=runs, dtype=float)
for grand_id, dct in results.items():
    for run, iterations in dct.items():
        summary.loc[grand_id, run] = iterations.KGE.max()

In [ ]:
# compare performance
fig, ax = plt.subplots(figsize=(8, 5))
sns.boxplot(
    summary,
    width=.5,
    showcaps=False,
    showfliers=False,
    boxprops=dict(facecolor='none', edgecolor='k', linewidth=1),
    whiskerprops=dict(color='k', linewidth=1),
    medianprops=dict(color='k', linewidth=2),
    zorder=2,
    ax=ax
)
sns.swarmplot(summary, ax=ax, zorder=1)
ax.set(
    ylim=(-1.05, 1.05),
    ylabel='KGE',
    title='CaMa-Flood',
)
ax.spines['left'].set_bounds([-1, 1])
ax.spines[['top', 'right', 'bottom']].set_visible(False)
ax.tick_params(axis='x', length=0);

### Compare convergence criteria

In [ ]:
compare_convergence = pd.concat([summary_old.median(), summary.median()], axis=1)
compare_convergence.columns = ['old', 'new']
compare_convergence

#### T-test

In [ ]:
p_values = {}
for run in runs:
    aux = pd.concat([summary_old[run], summary[run]], axis=1)
    aux.columns = ['old', 'new']
    aux.dropna(axis=0, how='any', inplace=True)
    _, p = ttest_rel(aux.old, aux.new)
    p_values[run] = p
    print(f'{run:>8} | p-value = {p:.3f}')

The T-test proves that there's a **significant improvement in performance with the new convergence criteria**, even though the median KGE improvements are in the order of 0.01.

### Compare calibration setups

In [ ]:
pp = sns.pairplot(
    summary,
    corner=True,
)

vlim = (-1, 1)
for ax in pp.axes.flat:
    if ax is not None: # Check if the axis object exists (corner=True leaves some empty)
        ax.plot(vlim, vlim, c='k', ls='--', lw=.5, zorder=0)
        ax.set(
            xlim=vlim,
            ylim=vlim
            )

#### T-test

In [ ]:
# List of all pairwise comparisons
cols = summary.columns.tolist()
pairs = list(combinations(cols, 2))

# Perform all paired t-tests
p_values = []
for (col1, col2) in pairs:
    aux = summary[[col1, col2]].copy()
    aux.dropna(axis=0, how='any', inplace=True)
    _, p = ttest_rel(aux[col1], aux[col2])
    p_values.append(p)

# Apply Bonferroni correction
reject, pvals_corrected, _, _ = multipletests(p_values, method='bonferroni')

# Show results
for i, (col1, col2) in enumerate(pairs):
    print(f"{col1:>8} vs {col2:>8} | p = {p_values[i]:.4f} | corrected p = {pvals_corrected[i]:.4f} | significant: {reject[i]}")


It seems like the only significant change in performance is due to the increase in the number of maximum iterations from 1000 to 2000 (or higher). No significance was found when comparing calibrations with 4 or 8 complexes, neither for the 1000-iteration nor the 2000-iteration setups.

From this results, I select the setup with **2000 maximum iterations and 4 complexes** for the calibration of the reservoir modules.

### Bivariate calibration

In this section I will test if the bivariate KGE is the best metric to calibrate both variables (outflow and storage) simultaneously.

I will rerun all the iterations in the calibration results to compute individually the KGE in terms of storage and outflow, and create a plot of the Pareto front and the highest bivariate KGE.

In [ ]:
run = '2000_4'

for grand_id, obs in tqdm(timeseries.items()):

    # extract samples from the calibration results
    samples = results[grand_id][run].copy()
    if ('KGE_storage' in samples.columns) & ('KGE_outflow' in samples.columns):
        continue
    samples[['KGE_storage', 'KGE_outflow']] = np.nan

    # define input time series
    inflow = obs[cfg.INFLOW]
    precipitation = obs[cfg.PRECIPITATION] if cfg.PRECIPITATION in obs.columns else None
    evaporation = obs[cfg.EVAPORATION] if cfg.EVAPORATION in obs.columns else None
    demand = obs[cfg.DEMAND] if cfg.DEMAND in obs.columns else None
    if cfg.MODEL == 'mhm':
        bias = obs.outflow.mean() / inflow.mean()
        demand = create_demand(
            obs.outflow,
            water_stress=min(1, bias),
            window=28
        )

    # attributes
    cap_cm, catch_sm, area_sm = attributes.loc[grand_id, ['CAP_MCM', 'CATCH_SKM', 'AREA_SKM']] * 1e6
    cap_cm = obs.storage.max()
    Vmin = max(0, min(0.1 * cap_cm, obs.storage.min()))
    Qmin = max(0, obs.outflow.min())

    calibrator = get_calibrator(
        cfg.MODEL,
        inflow=inflow,
        storage=obs.storage,
        outflow=obs.outflow,
        precipitation=precipitation,
        evaporation=evaporation,
        demand=demand,
        Vmin=Vmin,
        Vtot=cap_cm,
        Qmin=Qmin,
        Atot=area_sm,
        target=cfg.TARGET,
        obj_func=KGEmod,
        spinup=cfg.SPINUP,
        catchment=catch_sm
    )

    for i in tqdm(samples.index):
        # declare the reservoir model
        parameters = samples.loc[i, ['alpha', 'beta', 'gamma', 'delta', 'epsilon']].values
        calibrated_attrs = calibrator.pars2attrs(parameters)
        model = get_model(cfg.MODEL, **calibrated_attrs)
    
        # simulate
        Vo = obs.storage.iloc[0]
        sim = model.simulate(
            inflow=inflow,
            Vo=None if pd.isna(Vo) else Vo,
            precipitation=precipitation,
            evaporation=evaporation,
            demand=demand,
        )
    
        # performance
        for var in ['storage', 'outflow']:
            samples.loc[i, f'KGE_{var}'] = KGEmod(obs[var], sim[var])[0]

    # plot pareto front
    plot_pareto_front(
        samples.KGE_storage, 
        samples.KGE_outflow,
        title=grand_id,
        save=path_tests / run / subpath_calib / f'{grand_id}_pareto_front.jpg'
    )

    # export CSV
    samples.to_csv(path_tests / run / subpath_calib / f'{grand_id}_samples.csv', float_format='%.6f')

  0%|          | 0/164 [00:00<?, ?it/s]

  0%|          | 0/396 [00:00<?, ?it/s]

timesteps:   0%|          | 0/8400 [00:00<?, ?it/s]

timesteps:   0%|          | 0/8400 [00:00<?, ?it/s]

timesteps:   0%|          | 0/8400 [00:00<?, ?it/s]

...

timesteps:   0%|          | 0/6164 [00:00<?, ?it/s]

timesteps:   0%|          | 0/6164 [00:00<?, ?it/s]

timesteps:   0%|          | 0/6164 [00:00<?, ?it/s]

timesteps:   0%|          | 0/6164 [00:00<?, ?it/s]

timesteps:   0%|          | 0/6164 [00:00<?, ?it/s]

In [ ]:
samples.head()

In [ ]:
best_iter = samples.like1.idxmin()
samples.loc[[best_iter]]

In [ ]:
for i in tqdm([best_iter]):
    # declare the reservoir model
    parameters = samples.loc[i, ['alpha', 'beta', 'gamma', 'delta', 'epsilon']].values
    calibrated_attrs = calibrator.pars2attrs(parameters)
    model = get_model(cfg.MODEL, **calibrated_attrs)

    # simulate
    Vo = obs.storage.iloc[0]
    sim = model.simulate(
        inflow=inflow,
        Vo=None if pd.isna(Vo) else Vo,
        precipitation=precipitation,
        evaporation=evaporation,
        demand=demand,
    )

    model.lineplot(
        {'sim': sim},
        obs,
        spinup=cfg.SPINUP,
        figsize=(12, 6),
        # save=cfg.PATH_CALIB / f'{grand_id}_line.jpg',
    )


In [ ]:
pareto_front = is_pareto_efficient(samples.KGE_storage, samples.KGE_outflow)
front = samples.loc[pareto_front].sort_values('KGE_storage')

In [ ]:
from typing import Optional, Union
from lisfloodreservoirs.utils.metrics import is_pareto_efficient

In [ ]:
plot_pareto_front(
    samples.KGE_storage, 
    samples.KGE_outflow,
    title=grand_id,
    save=path_tests / run / subpath_calib / f'{grand_id}_pareto_front.jpg'
)

In [ ]:
path_tests / run / subpath_calib / f'{grand_id}_pareto_front.jpg'

In [ ]:
fig, ax = plt.subplots(figsize=(6, 5))
sct = ax.scatter(
    samples.KGE_storage, 
    samples.KGE_outflow, 
    c=samples.KGE_2var, 
    cmap='coolwarm_r', 
    vmin=-1, 
    vmax=1,
    s=4, 
    alpha=.7
)
ax.plot(front['KGE_storage'], front['KGE_outflow'], c='k', lw=.8, zorder=2)
ax.scatter(
    samples.loc[best_iter, 'KGE_storage'],
    samples.loc[best_iter, 'KGE_outflow'],
    marker='+',
    c='k',
    zorder=3
)
vlim = (-.025, 1.025)
ax.plot(vlim, vlim, c='k', ls='--', lw=.5)
ax.set(
    xlim=vlim,
    xlabel='KGE storage',
    ylim=vlim,
    ylabel='KGE outflow',
    title=grand_id,
)
plt.colorbar(sct, label='KGE bivariate', shrink=.66);

In [ ]:
kge_2var = 1 - np.sqrt((1 - kge_storage)**2 + (1 - kge_outflow)**2)

In [ ]:
kge_storage, kge_outflow, kge_2var

In [ ]:
1 - calibrator.objectivefunction(sim, obs)

In [ ]:
samples

In [ ]:
samples.loc[i, 'KGE']

In [ ]:
reservoir = Camaflood( 
    Vmin=Vmin,
    Vf=,
    Ve,
    Vtot,
    Qn,
    Qf=,
    catchment=catch_sm,
    Atot=area_sm,
    timestep: int = 86400
)